In [1]:
from pydbsmgr import *
from pydbsmgr.utils.tools import *
from pydbsmgr.health import FrameCheck
import pandas as pd

## Example of cleaning a dataset

In [2]:
# https://www.kaggle.com/datasets/amaanansari09/most-streamed-songs-all-time
features_df = pd.read_csv("./data/Features.csv", encoding="utf-8")
streams_df = pd.read_csv("./data/Streams.csv", encoding="utf-8")

list_of_dfs = [features_df, streams_df]
list_of_names = ["Features", "Streams"]

In [3]:
handler = FrameCheck(list_of_dfs, list_of_names)
handler.fix()
clean_dfs = handler.get_frames()
clean_dfs[0].head()

2024-01-19 00:51:57.724 | INFO     | pydbsmgr.health:fix:48 - 1) Empty columns have been removed.
2024-01-19 00:51:57.744 | INFO     | pydbsmgr.health:fix:50 - 1) Columns have been cleaned and transformed.
2024-01-19 00:51:57.763 | INFO     | pydbsmgr.health:_ops_dtypes:161 - 1) The data type has been verified.
2024-01-19 00:51:57.765 | INFO     | pydbsmgr.health:_ops_dtypes:163 - 1) The `nan` strings have been replaced by `np.nan`.
2024-01-19 00:51:57.769 | INFO     | pydbsmgr.health:_ops_dtypes:165 - 1) Only the named columns have been retained.
2024-01-19 00:51:57.773 | INFO     | pydbsmgr.health:fix:48 - 2) Empty columns have been removed.
2024-01-19 00:51:57.777 | INFO     | pydbsmgr.health:fix:50 - 2) Columns have been cleaned and transformed.
2024-01-19 00:51:57.933 | INFO     | pydbsmgr.health:_ops_dtypes:161 - 2) The data type has been verified.
2024-01-19 00:51:57.933 | INFO     | pydbsmgr.health:_ops_dtypes:163 - 2) The `nan` strings have been replaced by `np.nan`.
2024-01-1

,id,name,duration,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,danceability
0,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,3.33,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,0.514
1,7qiZfU4dY1lWllzX7mPBI3,Shape Of You,3.90,0.652,1,-3.183,0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,0.825
2,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,3.49,0.588,6,-6.400,0,0.0924,0.69200,0.000104,0.1490,0.513,98.027,0.824
3,7qEHsqek33rTcFNT9PFqLf,Someone You Loved,3.04,0.405,1,-5.679,1,0.0319,0.75100,0.000000,0.1050,0.446,109.891,0.501
4,0e7ipj03S05BNilyu5bRzt,Rockstar,3.64,0.520,5,-6.136,0,0.0712,0.12400,0.000070,0.1310,0.129,159.801,0.585


In [4]:
%%capture
handler.generate_report()

2024-01-19 00:52:01.293 | INFO     | pydbsmgr.health:generate_report:142 - DataFrame 'Features' has been processed
2024-01-19 00:52:03.553 | INFO     | pydbsmgr.health:generate_report:142 - DataFrame 'Streams' has been processed
2024-01-19 00:52:03.563 | INFO     | pydbsmgr.health:generate_report:146 - A report has been created under the name './report.html'


## ETL example using Azure Cloud 

In [ ]:
from datetime import date

from pydbsmgr.logs import *
from pydbsmgr.utils.azure_sdk import *

In [ ]:
connection_string = get_connection_string()
database_name = "test_database"
logsbook_name = "test_logsbook"
current_datetime = date.today()
container_name = "raw"

In [ ]:
# For this example, assume you’re using a dictionary to pass the data
data = {
    "name": ["Xavier", "Ann", "Jana", "Yi", "Robin", "Amal", "Nori"],
    "city": ["Mexico City", "Toronto", "Prague", "Shanghai", "Manchester", "Cairo", "Osaka"],
    "age": [41, 28, 33, 34, 38, 31, 37],
    "py-score": [88.0, 79.0, 81.0, 80.0, 68.0, 61.0, 84.0],
}

# Now you’re ready to create a pandas DataFrame
df = pd.DataFrame(data)

controller = StorageController(connection_string, container_name)
logbook = EventLogBook(logsbook_name, "./")

logbook_data = pd.DataFrame(
    {
        "container": [container_name],
        "name": [database_name],
        "datetime": [current_datetime],
    }
)

In [ ]:
%%capture
controller.upload_parquet("/", [df], [database_name])
controller.upload_parquet("/", [df], [database_name], compression=False)
BlobList = controller.get_BlobList("/")
controller._print_BlobPrefix()

del BlobList[0]

controller.set_BlobPrefix(BlobList)
controller._print_BlobPrefix()

dfs, names = controller.get_parquet("/", "\w+.parquet", True)

logbook.create(logbook_data)